In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

# Load and preprocess your dataset (assuming you've loaded the dataset with skipped column names)
data = pd.read_csv(r'C:\Users\grguo\Dropbox\000000000000000learning\COMP4730\Assignment 1\higgs10k.csv', header=None, skiprows=1)

# Define the number of folds for cross-validation
num_folds = 10

# Extract the label and features from the dataset
X = data.iloc[:, 1:]  # Features: All columns except the first
y = data.iloc[:, 0]   # Label: First column

# Create a K-Fold cross-validation object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a range of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20],     # Maximum depth of trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required to be in a leaf node
    'max_features': ['sqrt', 0.5],  # Use 'sqrt' or any valid float in (0.0, 1.0]
    'random_state': [42]  # Random seed for reproducibility
}

# Initialize a Random Forest classifier
rf = RandomForestClassifier()

# Create a GridSearchCV instance to search for the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=kf, scoring='accuracy')

# Initialize variables to store metrics
cross_val_accuracies = []
cross_val_recalls = []
cross_val_aucs = []

# Perform 10-fold cross-validation with hyperparameter tuning
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Perform hyperparameter tuning using GridSearchCV on the training data
    grid_search.fit(X_train, y_train)
    
    # Get the best model with tuned hyperparameters
    best_rf = grid_search.best_estimator_

    # Predict on the test data
    y_pred = best_rf.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    cross_val_accuracies.append(accuracy)

    # Calculate recall if there are positive samples in the test set
    if 1 in y_test.unique():
        recall = recall_score(y_test, y_pred)
        cross_val_recalls.append(recall)

    # Calculate AUC if the model has a predict_proba method
    if hasattr(best_rf, 'predict_proba'):
        y_prob = best_rf.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_prob)
        cross_val_aucs.append(auc)

# Calculate the average metrics
average_accuracy = np.mean(cross_val_accuracies)
average_recall = np.nan if len(cross_val_recalls) == 0 else np.mean(cross_val_recalls)
average_auc = np.nan if len(cross_val_aucs) == 0 else np.mean(cross_val_aucs)

# Print the results
print("Random Forest 10-Fold Cross-Validation Accuracy:", average_accuracy)
print("Random Forest 10-Fold Cross-Validation Recall:", average_recall)
if not np.isnan(average_auc):
    print("Random Forest 10-Fold Cross-Validation AUC:", average_auc)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:", grid_search.best_params_)
